In [1]:
import openai

client = openai.Client()

In [2]:
examples = [
    ("🚀 Just launched our biggest update yet! Try it now and let us know what you think!", "RED"),
    ("The server maintenance window is scheduled for tonight from 11pm to 1am.", "BLUE"),
    ("You won't believe what we just built! This is going to change everything. Check it out!", "RED"),
    ("All employees must complete the safety training module by Friday.", "BLUE"),
    ("🎉 We hit 500K users! Thank you to everyone who's been part of this journey!", "RED"),
    ("The quarterly financial report has been uploaded to the shared drive.", "BLUE"),
    ("Want to save 3 hours every day? Here's the secret workflow that changed my life!", "RED"),
    ("Reminder: Parking lot will be repaved this weekend. Please move your vehicles.", "BLUE"),
    ("🔥 Breaking: Our new AI feature is live! What do you think? Share your thoughts!", "RED"),
    ("The company picnic is scheduled for next Saturday at Central Park.", "BLUE"),
    ("This one trick will revolutionize how you work. Curious? Read on!", "RED"),
    ("All department heads should submit their budget proposals by end of month.", "BLUE"),
    ("💡 Pro tip: This simple change increased my productivity by 200%. Want to know how?", "RED"),
    ("The office will be closed on Monday for the holiday.", "BLUE"),
    ("We just released something amazing! Have you tried it? What's your experience?", "RED"),
]
classes = list(set(cls for _, cls in examples))
print(classes)

['RED', 'BLUE']


In [3]:
def llm(prompt):
    resp = client.chat.completions.create(model="gpt-4o", messages=[{"role":"user", "content": prompt}])
    return resp.choices[0].message.content

In [23]:
def evaluate(prompt, text):
    res = llm(prompt.format(text=text))
    for cls in classes:
        if cls in res.upper():
            return cls
    return "???"